# Create Sample Data

In [13]:
# Import dependencies
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine

# Generate 60 days of data
dates = [datetime.now() - timedelta(days=x) for x in range(60)]
regions = ['North', 'South', 'East', 'West']

data = []
for date in dates:
    for region in regions:
        total_orders = np.random.randint(50, 200)
        fulfilled_orders = int(total_orders * np.random.uniform(0.7, 0.99))
        avg_delivery_time = round(np.random.uniform(1.0, 3.5), 1)
        returns = np.random.randint(0, 10)
        inventory_level = np.random.randint(20, 100)
        
        data.append([
            date.date(),
            region,
            total_orders,
            fulfilled_orders,
            avg_delivery_time,
            returns,
            inventory_level
        ])

df = pd.DataFrame(data, columns=[
    'date', 'region', 'total_orders', 'fulfilled_orders', 
    'avg_delivery_time', 'returns', 'inventory_level'
])

# Save the data
df.to_csv('data/operational_data.csv', index=False)
print("Sample data generated successfully!")

Sample data generated successfully!


# Setup SQL Database

In [16]:
# Read CSV
df = pd.read_csv('data/operational_data.csv')
df.head()

,date,region,total_orders,fulfilled_orders,avg_delivery_time,returns,inventory_level
0,2025-06-03,North,63,58,3.2,2,25
1,2025-06-03,South,164,140,1.9,6,51
2,2025-06-03,East,66,63,1.3,1,49
3,2025-06-03,West,127,122,3.0,5,78
4,2025-06-02,North,74,65,3.1,2,85


In [18]:
# Create SQLite database
engine = create_engine('sqlite:///data/operations.db')

# Load data into database
df.to_sql('operations', engine, if_exists='replace', index=False)
print("Database created and data loaded successfully!")

Database created and data loaded successfully!


# Core Python Scripts

In [21]:
# Extract, transform and load
def extract_data():
    """Extract data from SQL database"""
    engine = create_engine('sqlite:///data/operations.db')
    query = """
    SELECT 
        date, region, 
        total_orders, fulfilled_orders,
        avg_delivery_time, returns, inventory_level
    FROM operations
    WHERE date >= date('now', '-7 days')
    """
    return pd.read_sql(query, engine)

def transform_data(df):
    """Transform and analyze data"""
    # Calculate metrics
    df['fulfillment_rate'] = df['fulfilled_orders'] / df['total_orders']
    df['return_rate'] = df['returns'] / df['total_orders']
    
    # Flag underperforming regions
    df['is_underperforming'] = (df['fulfillment_rate'] < 0.85) | \
                               (df['avg_delivery_time'] > 2.5) | \
                               (df['return_rate'] > 0.1)
    
    return df

def generate_report(df):
    """Generate HTML report"""
    # Summary stats
    summary = df.groupby('region').agg({
        'total_orders': 'sum',
        'fulfillment_rate': 'mean',
        'avg_delivery_time': 'mean',
        'return_rate': 'mean'
    }).reset_index()
    
    # Create HTML
    report = f"""
    <h1>Daily Operations Report - {datetime.now().date()}</h1>
    <h2>Regional Performance Summary</h2>
    {summary.to_html()}
    <h2>Underperforming Regions</h2>
    {df[df['is_underperforming']].to_html()}
    """
    
    with open('reports/daily_report.html', 'w') as f:
        f.write(report)
    
    return report

if __name__ == "__main__":
    # For testing
    df = extract_data()
    df = transform_data(df)
    generate_report(df)